In [ ]:
import pandas as pd
import random
import numpy as np
#!pip install nltk
import nltk
import string
#nltk.download()
#!pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer 
#nltk.download('wordnet')
import unicodedata
#!pip install inflect
import inflect

Error: Error: Activating Python 3.7.3 64-bit to run Jupyter failed with Error: Command failed: C:/Users/sosfr/Anaconda3/Scripts/activate && conda activate base && echo 'e8b39361-0157-4923-80e1-22d70d46dee6' && python c:/Users/sosfr/.vscode/extensions/ms-python.python-2019.11.50794/pythonFiles/printEnvVariables.py
The process cannot access the file because it is being used by another process.
The process cannot access the file because it is being used by another process.
The system cannot find the file C:\Users\sosfr\AppData\Local\Temp\conda-25500-3929.tmp.
The process cannot access the file because it is being used by another process.
'conda' is not recognized as an internal or external command,
operable program or batch file.
.

In [68]:
#Loading and Cleaning Data

#load in the csv contatining the data
df_original = pd.read_csv('wiki_movie_plots_deduped.csv')

In [69]:
#drop all rows in the dataframe that do not contain horror(1167) in genre catergory
df_horror = df_original.drop(df_original[df_original['Genre'] != 'horror'].index,inplace=False)
#drop columns we wont need
df_horror.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, 
               inplace = True)

#reduce horror to 1000 rows
df_horror = df_horror.sample(frac=1)
df_horror = df_horror.reset_index(drop=True)
df_horror_train = df_horror[0:1000]
df_horror_test = df_horror[1000:len(df_horror)]

In [70]:
#drop all rows in dataframe that do not contain comedy(4379)
df_comedy = df_original.drop(df_original[df_original['Genre'] != 'comedy'].index, inplace = False)
df_comedy.drop(['Release Year','Origin/Ethnicity','Cast', 'Wiki Page','Director','Release Year'],axis=1, 
               inplace = True)
#reduce horror to 1000 rows
df_comedy = df_comedy.sample(frac=1)
df_comedy = df_comedy.reset_index(drop=True)
df_comedy_train = df_comedy[0:1000]
df_comedy_test = df_comedy[1000:len(df_comedy)]

In [71]:
#merge the comedy and horror dataframes into one
df_HandC = pd.concat([df_horror_test,df_comedy_test])

#Pre punctuation parsing test
#df_HandC.iloc[1002]['Plot']

In [1]:
#Function to get rid of punctuation
def normalizeText(row):
    #create a list of punctuations we wish to delete
    punctuations = string.punctuation
    #change all letters to lower case
    row=row.lower()
    #loop through each word and get rid of punctuation and \r\n that came with data
    for x in row: 
        lemmatizer = WordNetLemmatizer()
        lemmatizer.lemmatize(x)
        if x in punctuations: 
            row = row.replace(x, "")
        row = row.replace('\r\n', '')
    return row
#run the column through the normalizing function
df_HandC["Plot"] = df_HandC["Plot"].apply(normalizeText) 

NameError: name 'df_HandC' is not defined

In [0]:
# Remove accented characters.
# Param: entire string will be converted, punctuation accepted
def removeAccents(string):
    return unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [86]:
# Replace numbers with word representation.
# NOTE: run before removing punctuation, result will have punctuation
# Param: single num, no punctuation or spaces within the string
def numToWord(num):
    try:
        float(num)
        return inflect.engine().number_to_words(num)
    except ValueError:
        return

In [75]:
#Post Punctuation Parsing test
#df_HandC.iloc[1002]['Plot']

In [112]:
##Prabesh being stupid. might be helpful?

#vectorizer = TfidfVectorizer()
#tfidf = vectorizer.fit_transform(df_HandC['Plot'])
#print similarity matrix
#print((tfidf*tfidf.T).A)

# Make list from df for "Plot" column
# Param: the df to use
from itertools import chain
def dfToList(df):
    lst = []
    for i, rows in df.iterrows():
        l = [rows.Plot]
        lst.append(l)
    return list(chain.from_iterable(lst))

# tf-idf vectorisation
# Param: all plots from the df as a list (ie [plot1, plot2,...])
def tfidfVec(plots):
    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(plots)
    feature_names = vectorizer.get_feature_names()
    dense = tfidf.todense()
    df_tfidf = pd.DataFrame(dense.tolist(), columns=feature_names)
    return df_tfidf

In [115]:
# test showing how it works on first 5 elements of df_HandC
tfidfVec(dfToList(df_HandC.head()))

,100,3s,about,abused,accepts,accidentally,act,actor,actress,admits,...,woman,womens,work,working,worry,would,wound,years,york,young
0,0.018012,0.018012,0.000000,0.018012,0.018012,0.000000,0.000000,0.018012,0.029065,0.018012,...,0.000000,0.018012,0.000000,0.018012,0.000000,0.014532,0.014532,0.018012,0.000000,0.00000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
2,0.000000,0.000000,0.066028,0.000000,0.000000,0.000000,0.000000,0.000000,0.066028,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.08184
3,0.000000,0.000000,0.051749,0.000000,0.000000,0.021381,0.021381,0.000000,0.000000,0.000000,...,0.042761,0.000000,0.021381,0.000000,0.021381,0.017250,0.017250,0.000000,0.021381,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000


In [83]:
##Prabesh being stupid. might be helpful?

#deleting the punctuation
without_punct = [w for w in df_HandC['Plot'] if w not in punctuation]
#deleting stopwords like the/we/for
sw = nltk.corpus.stopwords.words('english')
without_sw = [w for w in without_punct if w not in sw]
#lemmatizing the plot summary
wnl = nltk.WordNetLemmatizer()
lemmatizedSummary = wnl.lemmatize(without_sw)



TypeError: unhashable type: 'list'